# Scraping Hearts
#### _Web scraping mini project - Heart of Midlothian FC Fixtures and Results_

# Contents
## [Scraping the data](#scraping)
#### [Import the relevant packages](#import_packages)
#### [Define the base site](#)
#### [Send the request](#send_request)
#### [Examine HTML content](#examine_html)
#### [Create Beautiful Soup object](#soup_object)
#### [Export the HTML](#export_html)
#### [Locate information](#locate_info)
# [Extract and store the desired information](#extract)
#### [Date](#date)
##### [Convert to datetime object](#format_date)
#### [Competition](#competition)
#### [Home or Away](#home_away)
#### [Venue](#venue)
#### [Teams](#teams)
#### [Score](#score)
# [Create a pandas dataframe](#pandas_dataframe)
#### [Set index to match_day](#set_index)
# ['Feature Engineering': Use the Home and Away Goals to create a Win/Loss/Draw feature](#feature_engineering)
### [Data cleaning: Home and away goals](#home_away_cleaning)
#### [Convert Home and Away_team_goals from string to integer data type](#convert_data_type)
#### [Use a Regex to remove 'kick off' values](#regex)
#### [Create home and away goals columns](#create_home_away_columns)
#### [Create win-loss-draw columns](#win_loss_draw)
#### [Conditional logic: use _numpy.select_ for determing a win, loss or draw](#np_select)
#### [Drop, rename and reorder columns](#drop_column)
#### [Locate fixtures / venues](#locate_info)
#### [Add TBD values for matches that haven't been played yet](#tbd_data)
# [Export to _.csv_](#export_csv)
# [Explore the data frame and descriptives of the season so far](#explore)

------------------------------------------------
# Scraping the data
------------------------------------------------

### Import the packages <a id='import_packages' ><a/>

In [2]:
# import the packages
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import date
from dateutil.parser import parse
import re

### Defining the base site <a id='base_site' ><a/>

In [3]:
# define the url of the base site
base_site = "https://www.heartsfc.co.uk/fixtures/first-team/fixtures-and-results"

### Sending the request and checking response <a id='send_request' ><a/>

In [4]:
# send a request to the webpage and check its status
response = requests.get(base_site)
response.status_code

200

### Examine the html content <a id='examine_html' ><a/>

In [5]:
# get the html from the webpage
html = response.content

### Create a _BeautifulSoup_ object <a id='soup_object' ><a/>
* Creating a _BeautifulSoup_ object parses the _html_ data we have obtained from our request and enables us to extract the details we are looking for.

In [6]:
# create a BeautfulSoup object
soup = BeautifulSoup(html, 'html.parser')

### Export the _BeautifulSoup_ obejct as an HTML file so can explore it in Atom <a id='export_html' ><a/>
* Use _Atom_ to check the output to ensure that the html file parsed correctly. If it didn't work we'll need to use '_lxml_' as a parser when creating the _BeautifulSoup_ object.

In [7]:
# create an html file so we can inspect it
with open('Hearts_fixtures_and_results_HTML_parser.html', 'wb') as file:
    file.write(soup.prettify('utf-8'))

### Checking the output <a id='locate_info' ><a/>

In [8]:
# locate the div that contains the info we'll extract
divs = soup.findAll("div", {"class": "fixtureItem"}, limit=None)
divs

[<div class="fixtureItem">
 <div class="fixtureItemClass">
         A
     </div>
 <div class="fixtureItemInfo">
 <p>
 <small>Premier Sports Cup</small>
             Sat 10th July 2021
         </p>
 </div>
 <div class="fixtureItemDetails">
 <div class="fixtureItemDetailsInner">
 <div class="fixtureItemTeam">
 <div class="fixtureItemTeamLogo">
 <picture>
 <source )="" media="all and (max-width: 1024px)" srcset="/uploads/images/Clubs/teams/large/1622561529Peterheadbadge.png 1x, /uploads/images/Clubs/teams/large_2x/1622561529Peterheadbadge.png 2x"/>
 <img alt="Peterhead logo" srcset="/uploads/images/Clubs/teams/small/1622561529Peterheadbadge.png 1x, /uploads/images/Clubs/teams/small/1622561529Peterheadbadge.png 2x"/>
 </picture>
 </div>
 <span>Peterhead</span>
 </div>
 <div class="fixtureItemMeta">
 <small>FT:</small>
                     0 -  2
                             </div>
 <div class="fixtureItemTeam alt">
 <div class="fixtureItemTeamLogo">
 <picture>
 <source )="" media="all an

In [9]:
# examine the first div
divs[0]

<div class="fixtureItem">
<div class="fixtureItemClass">
        A
    </div>
<div class="fixtureItemInfo">
<p>
<small>Premier Sports Cup</small>
            Sat 10th July 2021
        </p>
</div>
<div class="fixtureItemDetails">
<div class="fixtureItemDetailsInner">
<div class="fixtureItemTeam">
<div class="fixtureItemTeamLogo">
<picture>
<source )="" media="all and (max-width: 1024px)" srcset="/uploads/images/Clubs/teams/large/1622561529Peterheadbadge.png 1x, /uploads/images/Clubs/teams/large_2x/1622561529Peterheadbadge.png 2x"/>
<img alt="Peterhead logo" srcset="/uploads/images/Clubs/teams/small/1622561529Peterheadbadge.png 1x, /uploads/images/Clubs/teams/small/1622561529Peterheadbadge.png 2x"/>
</picture>
</div>
<span>Peterhead</span>
</div>
<div class="fixtureItemMeta">
<small>FT:</small>
                    0 -  2
                            </div>
<div class="fixtureItemTeam alt">
<div class="fixtureItemTeamLogo">
<picture>
<source )="" media="all and (max-width: 1024px)" srcset

------------------------------------------------
# Extract the desired information and save in a variable <a id='extract' ><a/>
------------------------------------------------

## Date <a id='date' ><a/>

In [10]:
# extract date
match_date = [div.find("h5").string for div in divs]
match_date

['Saturday 10th July 2021',
 'Tuesday 13th July 2021',
 'Saturday 17th July 2021',
 'Tuesday 20th July 2021',
 'Sunday 25th July 2021',
 'Saturday 31st July 2021',
 'Saturday 7th August 2021',
 'Sunday 15th August 2021',
 'Sunday 22nd August 2021',
 'Saturday 28th August 2021',
 'Sunday 12th September 2021',
 'Saturday 18th September 2021',
 'Saturday 25th September 2021',
 'Saturday 2nd October 2021',
 'Saturday 16th October 2021',
 'Saturday 23rd October 2021',
 'Wednesday 27th October 2021',
 'Saturday 30th October 2021',
 'Saturday 6th November 2021',
 'Saturday 20th November 2021',
 'Saturday 27th November 2021',
 'Thursday 2nd December 2021',
 'Sunday 5th December 2021',
 'Sunday 12th December 2021',
 'Saturday 18th December 2021',
 'Sunday 26th December 2021',
 'Wednesday 29th December 2021',
 'Monday 3rd January 2022',
 'Wednesday 26th January 2022',
 'Saturday 29th January 2022',
 'Saturday 5th February 2022',
 'Wednesday 9th February 2022',
 'Saturday 19th February 2022',
 'S

### Format into _datetime.date_ <a id='format_date' ><a/>

In [11]:
# use dateutil.parser parse to turn dates into datetime data type
date_formatted = [parse(i) for i in match_date]
date_formatted

[datetime.datetime(2021, 7, 10, 0, 0),
 datetime.datetime(2021, 7, 13, 0, 0),
 datetime.datetime(2021, 7, 17, 0, 0),
 datetime.datetime(2021, 7, 20, 0, 0),
 datetime.datetime(2021, 7, 25, 0, 0),
 datetime.datetime(2021, 7, 31, 0, 0),
 datetime.datetime(2021, 8, 7, 0, 0),
 datetime.datetime(2021, 8, 15, 0, 0),
 datetime.datetime(2021, 8, 22, 0, 0),
 datetime.datetime(2021, 8, 28, 0, 0),
 datetime.datetime(2021, 9, 12, 0, 0),
 datetime.datetime(2021, 9, 18, 0, 0),
 datetime.datetime(2021, 9, 25, 0, 0),
 datetime.datetime(2021, 10, 2, 0, 0),
 datetime.datetime(2021, 10, 16, 0, 0),
 datetime.datetime(2021, 10, 23, 0, 0),
 datetime.datetime(2021, 10, 27, 0, 0),
 datetime.datetime(2021, 10, 30, 0, 0),
 datetime.datetime(2021, 11, 6, 0, 0),
 datetime.datetime(2021, 11, 20, 0, 0),
 datetime.datetime(2021, 11, 27, 0, 0),
 datetime.datetime(2021, 12, 2, 0, 0),
 datetime.datetime(2021, 12, 5, 0, 0),
 datetime.datetime(2021, 12, 12, 0, 0),
 datetime.datetime(2021, 12, 18, 0, 0),
 datetime.datetime

In [12]:
type(date_formatted[0])

datetime.datetime

In [13]:
date_formatted[0]

datetime.datetime(2021, 7, 10, 0, 0)

In [14]:
date_formatted[0].date()

datetime.date(2021, 7, 10)

In [15]:
date_formatted = [i.date() for i in date_formatted]
date_formatted

[datetime.date(2021, 7, 10),
 datetime.date(2021, 7, 13),
 datetime.date(2021, 7, 17),
 datetime.date(2021, 7, 20),
 datetime.date(2021, 7, 25),
 datetime.date(2021, 7, 31),
 datetime.date(2021, 8, 7),
 datetime.date(2021, 8, 15),
 datetime.date(2021, 8, 22),
 datetime.date(2021, 8, 28),
 datetime.date(2021, 9, 12),
 datetime.date(2021, 9, 18),
 datetime.date(2021, 9, 25),
 datetime.date(2021, 10, 2),
 datetime.date(2021, 10, 16),
 datetime.date(2021, 10, 23),
 datetime.date(2021, 10, 27),
 datetime.date(2021, 10, 30),
 datetime.date(2021, 11, 6),
 datetime.date(2021, 11, 20),
 datetime.date(2021, 11, 27),
 datetime.date(2021, 12, 2),
 datetime.date(2021, 12, 5),
 datetime.date(2021, 12, 12),
 datetime.date(2021, 12, 18),
 datetime.date(2021, 12, 26),
 datetime.date(2021, 12, 29),
 datetime.date(2022, 1, 3),
 datetime.date(2022, 1, 26),
 datetime.date(2022, 1, 29),
 datetime.date(2022, 2, 5),
 datetime.date(2022, 2, 9),
 datetime.date(2022, 2, 19),
 datetime.date(2022, 2, 26),
 datetim

## Competition <a id='competition' ><a/>

In [16]:
# extract the competition
divs[0].find("small").string

'Premier Sports Cup'

In [17]:
# make a list of competitions
competition = [div.find("small").string for div in divs]
competition

['Premier Sports Cup',
 'Premier Sports Cup',
 'Friendly',
 'Premier Sports Cup',
 'Premier Sports Cup',
 'cinch Premiership',
 'cinch Premiership',
 'Premier Sports Cup',
 'cinch Premiership',
 'cinch Premiership',
 'cinch Premiership',
 'cinch Premiership',
 'cinch Premiership',
 'cinch Premiership',
 'cinch Premiership',
 'cinch Premiership',
 'cinch Premiership',
 'cinch Premiership',
 'cinch Premiership',
 'cinch Premiership',
 'cinch Premiership',
 'cinch Premiership',
 'cinch Premiership',
 'cinch Premiership',
 'cinch Premiership',
 'cinch Premiership',
 'cinch Premiership',
 'cinch Premiership',
 'cinch Premiership',
 'cinch Premiership',
 'cinch Premiership',
 'cinch Premiership',
 'cinch Premiership',
 'cinch Premiership',
 'cinch Premiership',
 'cinch Premiership',
 'cinch Premiership',
 'cinch Premiership',
 'cinch Premiership']

## Home or Away Fixture <a id='home_away' ><a/>

In [18]:
# Home_or _Away fixture
divs[0].find("div", {"class": "fixtureItemClass"})

<div class="fixtureItemClass">
        A
    </div>

In [19]:
# list comprehension to extract a list of H's and A's
home_away = [div.find("div", {"class": "fixtureItemClass"}) for div in divs]
print(home_away)

[<div class="fixtureItemClass">
        A
    </div>, <div class="fixtureItemClass">
        H
    </div>, <div class="fixtureItemClass">
        H
    </div>, <div class="fixtureItemClass">
        A
    </div>, <div class="fixtureItemClass">
        H
    </div>, <div class="fixtureItemClass">
        H
    </div>, <div class="fixtureItemClass">
        A
    </div>, <div class="fixtureItemClass">
        A
    </div>, <div class="fixtureItemClass">
        H
    </div>, <div class="fixtureItemClass">
        A
    </div>, <div class="fixtureItemClass">
        H
    </div>, <div class="fixtureItemClass">
        A
    </div>, <div class="fixtureItemClass">
        H
    </div>, <div class="fixtureItemClass">
        H
    </div>, <div class="fixtureItemClass">
        A
    </div>, <div class="fixtureItemClass">
        H
    </div>, <div class="fixtureItemClass">
        A
    </div>, <div class="fixtureItemClass">
        A
    </div>, <div class="fixtureItemClass">
        H
    

In [20]:
# get the H or A on its own
home_away[0].text.lstrip().rstrip()

'A'

In [21]:
# list comprehension to get a list of jus the 'H' or 'A'
home_away = [div.find("div", {"class": "fixtureItemClass"}).text.lstrip().rstrip() for div in divs]
home_away

['A',
 'H',
 'H',
 'A',
 'H',
 'H',
 'A',
 'A',
 'H',
 'A',
 'H',
 'A',
 'H',
 'H',
 'A',
 'H',
 'A',
 'A',
 'H',
 'A',
 'H',
 'A',
 'A',
 'H',
 'A',
 'H',
 'H',
 'A',
 'H',
 'H',
 'A',
 'H',
 'A',
 'A',
 'H',
 'A',
 'H',
 'A',
 'H']

## Venue <a id='venue' ><a/>

In [22]:
# get venue
divs[0].find("div", {"class": "fixtureSmallInfo mobile"})

<div class="fixtureSmallInfo mobile">
<p>Premier Sports Cup</p>
<h5>Saturday 10th July 2021</h5>
<p>Balmoor</p>
</div>

In [23]:
divs[0].findAll("p")

[<p>
 <small>Premier Sports Cup</small>
             Sat 10th July 2021
         </p>,
 <p>Premier Sports Cup</p>,
 <p>Balmoor</p>]

In [24]:
divs[0].findAll("p")[2]

<p>Balmoor</p>

In [25]:
divs[0].findAll("p")[2].string

'Balmoor'

In [26]:
venue = [div.findAll("p")[2].string for div in divs]
venue

['Balmoor',
 'Tynecastle Park',
 'Tynecastle',
 'Forthbank',
 'Tynecastle Park',
 'Tynecastle Park',
 'St Mirren Park',
 'Celtic Park',
 'Tynecastle Park',
 'Tannadice',
 'Tynecastle Park',
 'Global Energy Stadium',
 'Tynecastle Park',
 'Tynecastle Park',
 'Ibrox',
 'Tynecastle Park',
 'McDiarmid Park',
 'Pittodrie',
 'Tynecastle Park',
 'Fir Park',
 'Tynecastle Park',
 'Celtic Park',
 'Tony Macaroni Arena',
 'Tynecastle Park',
 'Dens Park',
 'Tynecastle Park',
 'Tynecastle Park',
 'Easter Road',
 'Tynecastle Park',
 'Tynecastle Park',
 'Ibrox',
 'Tynecastle Park',
 'McDiarmid Park',
 'St Mirren Park',
 'Tynecastle Park',
 'Tannadice',
 'Tynecastle Park',
 'Global Energy Stadium',
 'Tynecastle Park']

## Home and Away teams <a id='teams' ><a/>

In [27]:
teams = [div.find_all("span") for div in divs]
teams

[[<span>Peterhead</span>, <span>Heart of Midlothian</span>],
 [<span>Heart of Midlothian</span>, <span>Cove Rangers</span>],
 [<span>Heart of Midlothian</span>, <span>Sunderland</span>],
 [<span>Stirling Albion</span>, <span>Heart of Midlothian</span>],
 [<span>Heart of Midlothian</span>, <span>Inverness CT</span>],
 [<span>Heart of Midlothian</span>, <span>Celtic</span>],
 [<span>St Mirren</span>, <span>Heart of Midlothian</span>],
 [<span>Celtic</span>, <span>Heart of Midlothian</span>],
 [<span>Heart of Midlothian</span>, <span>Aberdeen</span>],
 [<span>Dundee United</span>, <span>Heart of Midlothian</span>],
 [<span>Heart of Midlothian</span>, <span>Hibernian</span>],
 [<span>Ross County</span>, <span>Heart of Midlothian</span>],
 [<span>Heart of Midlothian</span>, <span>Livingston</span>],
 [<span>Heart of Midlothian</span>, <span>Motherwell</span>],
 [<span>Rangers</span>, <span>Heart of Midlothian</span>],
 [<span>Heart of Midlothian</span>, <span>Dundee</span>],
 [<span>St John

In [28]:
home_team = []
away_team = []

for k, v in teams:
    home_team.append(k)
    away_team.append(v)

In [29]:
home_team

[<span>Peterhead</span>,
 <span>Heart of Midlothian</span>,
 <span>Heart of Midlothian</span>,
 <span>Stirling Albion</span>,
 <span>Heart of Midlothian</span>,
 <span>Heart of Midlothian</span>,
 <span>St Mirren</span>,
 <span>Celtic</span>,
 <span>Heart of Midlothian</span>,
 <span>Dundee United</span>,
 <span>Heart of Midlothian</span>,
 <span>Ross County</span>,
 <span>Heart of Midlothian</span>,
 <span>Heart of Midlothian</span>,
 <span>Rangers</span>,
 <span>Heart of Midlothian</span>,
 <span>St Johnstone</span>,
 <span>Aberdeen</span>,
 <span>Heart of Midlothian</span>,
 <span>Motherwell</span>,
 <span>Heart of Midlothian</span>,
 <span>Celtic</span>,
 <span>Livingston</span>,
 <span>Heart of Midlothian</span>,
 <span>Dundee</span>,
 <span>Heart of Midlothian</span>,
 <span>Heart of Midlothian</span>,
 <span>Hibernian</span>,
 <span>Heart of Midlothian</span>,
 <span>Heart of Midlothian</span>,
 <span>Rangers</span>,
 <span>Heart of Midlothian</span>,
 <span>St Johnstone</span>,

In [30]:
away_team

[<span>Heart of Midlothian</span>,
 <span>Cove Rangers</span>,
 <span>Sunderland</span>,
 <span>Heart of Midlothian</span>,
 <span>Inverness CT</span>,
 <span>Celtic</span>,
 <span>Heart of Midlothian</span>,
 <span>Heart of Midlothian</span>,
 <span>Aberdeen</span>,
 <span>Heart of Midlothian</span>,
 <span>Hibernian</span>,
 <span>Heart of Midlothian</span>,
 <span>Livingston</span>,
 <span>Motherwell</span>,
 <span>Heart of Midlothian</span>,
 <span>Dundee</span>,
 <span>Heart of Midlothian</span>,
 <span>Heart of Midlothian</span>,
 <span>Dundee United</span>,
 <span>Heart of Midlothian</span>,
 <span>St Mirren</span>,
 <span>Heart of Midlothian</span>,
 <span>Heart of Midlothian</span>,
 <span>Rangers</span>,
 <span>Heart of Midlothian</span>,
 <span>Ross County</span>,
 <span>St Johnstone</span>,
 <span>Heart of Midlothian</span>,
 <span>Celtic</span>,
 <span>Motherwell</span>,
 <span>Heart of Midlothian</span>,
 <span>Dundee</span>,
 <span>Heart of Midlothian</span>,
 <span>Hear

In [31]:
home_team = [i.string for i in home_team]
home_team

['Peterhead',
 'Heart of Midlothian',
 'Heart of Midlothian',
 'Stirling Albion',
 'Heart of Midlothian',
 'Heart of Midlothian',
 'St Mirren',
 'Celtic',
 'Heart of Midlothian',
 'Dundee United',
 'Heart of Midlothian',
 'Ross County',
 'Heart of Midlothian',
 'Heart of Midlothian',
 'Rangers',
 'Heart of Midlothian',
 'St Johnstone',
 'Aberdeen',
 'Heart of Midlothian',
 'Motherwell',
 'Heart of Midlothian',
 'Celtic',
 'Livingston',
 'Heart of Midlothian',
 'Dundee',
 'Heart of Midlothian',
 'Heart of Midlothian',
 'Hibernian',
 'Heart of Midlothian',
 'Heart of Midlothian',
 'Rangers',
 'Heart of Midlothian',
 'St Johnstone',
 'St Mirren',
 'Heart of Midlothian',
 'Dundee United',
 'Heart of Midlothian',
 'Ross County',
 'Heart of Midlothian']

In [32]:
away_team = [i.string for i in away_team]
away_team

['Heart of Midlothian',
 'Cove Rangers',
 'Sunderland',
 'Heart of Midlothian',
 'Inverness CT',
 'Celtic',
 'Heart of Midlothian',
 'Heart of Midlothian',
 'Aberdeen',
 'Heart of Midlothian',
 'Hibernian',
 'Heart of Midlothian',
 'Livingston',
 'Motherwell',
 'Heart of Midlothian',
 'Dundee',
 'Heart of Midlothian',
 'Heart of Midlothian',
 'Dundee United',
 'Heart of Midlothian',
 'St Mirren',
 'Heart of Midlothian',
 'Heart of Midlothian',
 'Rangers',
 'Heart of Midlothian',
 'Ross County',
 'St Johnstone',
 'Heart of Midlothian',
 'Celtic',
 'Motherwell',
 'Heart of Midlothian',
 'Dundee',
 'Heart of Midlothian',
 'Heart of Midlothian',
 'Aberdeen',
 'Heart of Midlothian',
 'Livingston',
 'Heart of Midlothian',
 'Hibernian']

## Full-time Score <a id='score' ><a/>

In [33]:
# find score
divs[0].find("div", {"class": "fixtureItemMeta"}).text.replace('FT:\n', '').rstrip().lstrip()

'0 -  2'

In [34]:
# get all the scores
scores = [div.find("div", {"class": "fixtureItemMeta"}).text.replace('FT:\n', '').rstrip().lstrip() for div in divs]
scores

['0 -  2',
 '3 -  0',
 '0 -  2',
 '0 -  2',
 '1 -  0',
 '2 -  1',
 '1 -  2',
 '3 -  2',
 '1 -  1',
 '0 -  2',
 '0 -  0',
 '2 -  2',
 '3 -  0',
 '2 -  0',
 '1 -  1',
 '1 -  1',
 '1 -  1',
 '2 -  1',
 'KO:\n                    15:00',
 'KO:\n                    15:00',
 'KO:\n                    15:00',
 'KO:\n                    19:45',
 'KO:\n                    15:00',
 'KO:\n                    12:00',
 'KO:\n                    15:00',
 'KO:\n                    15:00',
 'KO:\n                    19:45',
 'KO:\n                    15:00',
 'KO:\n                    19:45',
 'KO:\n                    15:00',
 'KO:\n                    15:00',
 'KO:\n                    19:45',
 'KO:\n                    15:00',
 'KO:\n                    15:00',
 'KO:\n                    19:45',
 'KO:\n                    15:00',
 'KO:\n                    15:00',
 'KO:\n                    15:00',
 'KO:\n                    15:00']

In [35]:
# remove the \n from the output
scores = [i.replace('\n', '') for i in scores]
scores

['0 -  2',
 '3 -  0',
 '0 -  2',
 '0 -  2',
 '1 -  0',
 '2 -  1',
 '1 -  2',
 '3 -  2',
 '1 -  1',
 '0 -  2',
 '0 -  0',
 '2 -  2',
 '3 -  0',
 '2 -  0',
 '1 -  1',
 '1 -  1',
 '1 -  1',
 '2 -  1',
 'KO:                    15:00',
 'KO:                    15:00',
 'KO:                    15:00',
 'KO:                    19:45',
 'KO:                    15:00',
 'KO:                    12:00',
 'KO:                    15:00',
 'KO:                    15:00',
 'KO:                    19:45',
 'KO:                    15:00',
 'KO:                    19:45',
 'KO:                    15:00',
 'KO:                    15:00',
 'KO:                    19:45',
 'KO:                    15:00',
 'KO:                    15:00',
 'KO:                    19:45',
 'KO:                    15:00',
 'KO:                    15:00',
 'KO:                    15:00',
 'KO:                    15:00']

In [36]:
# remove the blank sapces
scores = [i.replace(' ', '') for i in scores]
scores

['0-2',
 '3-0',
 '0-2',
 '0-2',
 '1-0',
 '2-1',
 '1-2',
 '3-2',
 '1-1',
 '0-2',
 '0-0',
 '2-2',
 '3-0',
 '2-0',
 '1-1',
 '1-1',
 '1-1',
 '2-1',
 'KO:15:00',
 'KO:15:00',
 'KO:15:00',
 'KO:19:45',
 'KO:15:00',
 'KO:12:00',
 'KO:15:00',
 'KO:15:00',
 'KO:19:45',
 'KO:15:00',
 'KO:19:45',
 'KO:15:00',
 'KO:15:00',
 'KO:19:45',
 'KO:15:00',
 'KO:15:00',
 'KO:19:45',
 'KO:15:00',
 'KO:15:00',
 'KO:15:00',
 'KO:15:00']

In [37]:
# check the data type
type(scores[5])

str

------------------------------------------------
# Create a _pandas_ dataframe <a id='pandas_dataframe' ><a/>
------------------------------------------------

In [38]:
# create a dataframe
data = {'Date': match_date, 'Date_formatted': date_formatted, 'Competition': competition, 'Home_or_Away': home_away, 'Venue': venue, 
        'Home_team': home_team, 'Away_team': away_team, 'Score': scores}

df = pd.DataFrame(data)
df.head()

,Date,Date_formatted,Competition,Home_or_Away,Venue,Home_team,Away_team,Score
0,Saturday 10th July 2021,2021-07-10,Premier Sports Cup,A,Balmoor,Peterhead,Heart of Midlothian,0-2
1,Tuesday 13th July 2021,2021-07-13,Premier Sports Cup,H,Tynecastle Park,Heart of Midlothian,Cove Rangers,3-0
2,Saturday 17th July 2021,2021-07-17,Friendly,H,Tynecastle,Heart of Midlothian,Sunderland,0-2
3,Tuesday 20th July 2021,2021-07-20,Premier Sports Cup,A,Forthbank,Stirling Albion,Heart of Midlothian,0-2
4,Sunday 25th July 2021,2021-07-25,Premier Sports Cup,H,Tynecastle Park,Heart of Midlothian,Inverness CT,1-0


## Set index to '_Match_day\_(number)_' <a id='set_index' ><a/>

In [39]:
# create Match day 1-39
match_day = []
for i in range(0, len(df['Date'])):
    match_day.append(f"Match_day_{i+1}")

match_day

['Match_day_1',
 'Match_day_2',
 'Match_day_3',
 'Match_day_4',
 'Match_day_5',
 'Match_day_6',
 'Match_day_7',
 'Match_day_8',
 'Match_day_9',
 'Match_day_10',
 'Match_day_11',
 'Match_day_12',
 'Match_day_13',
 'Match_day_14',
 'Match_day_15',
 'Match_day_16',
 'Match_day_17',
 'Match_day_18',
 'Match_day_19',
 'Match_day_20',
 'Match_day_21',
 'Match_day_22',
 'Match_day_23',
 'Match_day_24',
 'Match_day_25',
 'Match_day_26',
 'Match_day_27',
 'Match_day_28',
 'Match_day_29',
 'Match_day_30',
 'Match_day_31',
 'Match_day_32',
 'Match_day_33',
 'Match_day_34',
 'Match_day_35',
 'Match_day_36',
 'Match_day_37',
 'Match_day_38',
 'Match_day_39']

In [40]:
df = df.set_axis(labels=match_day, axis=0)
df.head()

,Date,Date_formatted,Competition,Home_or_Away,Venue,Home_team,Away_team,Score
Match_day_1,Saturday 10th July 2021,2021-07-10,Premier Sports Cup,A,Balmoor,Peterhead,Heart of Midlothian,0-2
Match_day_2,Tuesday 13th July 2021,2021-07-13,Premier Sports Cup,H,Tynecastle Park,Heart of Midlothian,Cove Rangers,3-0
Match_day_3,Saturday 17th July 2021,2021-07-17,Friendly,H,Tynecastle,Heart of Midlothian,Sunderland,0-2
Match_day_4,Tuesday 20th July 2021,2021-07-20,Premier Sports Cup,A,Forthbank,Stirling Albion,Heart of Midlothian,0-2
Match_day_5,Sunday 25th July 2021,2021-07-25,Premier Sports Cup,H,Tynecastle Park,Heart of Midlothian,Inverness CT,1-0


------------------------------------------------
# 'Feature Engineering': Win/Loss/Draw <a id='feature_engineering' ><a/>
------------------------------------------------
* A column that indicates whether Hearts won, lost or drew would be very useful for analysis purposes. 
* It would also be crucial if this project were to be taken (much) further and used in a machine learning context.
* So, let's create a column and populate it with a 'W', 'L' or 'D' depending on the score...

## Data cleaning: Home and Away goals <a id='home_away_cleaning' ><a/>
**Step 1** is to tidy up the '_scores_' data so we can more easily extract the data we want from it.

In [41]:
# check the output
scores

['0-2',
 '3-0',
 '0-2',
 '0-2',
 '1-0',
 '2-1',
 '1-2',
 '3-2',
 '1-1',
 '0-2',
 '0-0',
 '2-2',
 '3-0',
 '2-0',
 '1-1',
 '1-1',
 '1-1',
 '2-1',
 'KO:15:00',
 'KO:15:00',
 'KO:15:00',
 'KO:19:45',
 'KO:15:00',
 'KO:12:00',
 'KO:15:00',
 'KO:15:00',
 'KO:19:45',
 'KO:15:00',
 'KO:19:45',
 'KO:15:00',
 'KO:15:00',
 'KO:19:45',
 'KO:15:00',
 'KO:15:00',
 'KO:19:45',
 'KO:15:00',
 'KO:15:00',
 'KO:15:00',
 'KO:15:00']

In [42]:
# split the scores to extract a list with 2 numbers in it: the first will be the home goals, the second will be the away goals
scores[0].split("-")

['0', '2']

In [43]:
type(scores)

list

In [44]:
type(scores[0])

str

## Create Home and Away team goals columns <a id='create_home_away_columns' ><a/>
**Step 2** is to create home and away goals columns so we can apply conditional logic to this data to create a win/loss/draw feature.

In [45]:
# create home and away goals column

# create empty lists to be populated
Home_team_goals = []
Away_team_goals = []

# loop through scores andextract the first character in the string (e.g. 0 from '0-2') as the home goal and the 3rd character as the Away_teams goals
for i in scores:
    if i[0] != 'K':
        Home_team_goals.append(i[0])
        Away_team_goals.append(i[2])
    elif i[0] == 'K':
        Home_team_goals.append(i)
        Away_team_goals.append(i)
        

In [46]:
# check the output
Home_team_goals

['0',
 '3',
 '0',
 '0',
 '1',
 '2',
 '1',
 '3',
 '1',
 '0',
 '0',
 '2',
 '3',
 '2',
 '1',
 '1',
 '1',
 '2',
 'KO:15:00',
 'KO:15:00',
 'KO:15:00',
 'KO:19:45',
 'KO:15:00',
 'KO:12:00',
 'KO:15:00',
 'KO:15:00',
 'KO:19:45',
 'KO:15:00',
 'KO:19:45',
 'KO:15:00',
 'KO:15:00',
 'KO:19:45',
 'KO:15:00',
 'KO:15:00',
 'KO:19:45',
 'KO:15:00',
 'KO:15:00',
 'KO:15:00',
 'KO:15:00']

In [47]:
# check the output
Away_team_goals

['2',
 '0',
 '2',
 '2',
 '0',
 '1',
 '2',
 '2',
 '1',
 '2',
 '0',
 '2',
 '0',
 '0',
 '1',
 '1',
 '1',
 '1',
 'KO:15:00',
 'KO:15:00',
 'KO:15:00',
 'KO:19:45',
 'KO:15:00',
 'KO:12:00',
 'KO:15:00',
 'KO:15:00',
 'KO:19:45',
 'KO:15:00',
 'KO:19:45',
 'KO:15:00',
 'KO:15:00',
 'KO:19:45',
 'KO:15:00',
 'KO:15:00',
 'KO:19:45',
 'KO:15:00',
 'KO:15:00',
 'KO:15:00',
 'KO:15:00']

In [48]:
# add the data to our dataframe and check the output
df['Home_team_goals'] = Home_team_goals
df['Away_team_goals'] = Away_team_goals
df.head()

,Date,Date_formatted,Competition,Home_or_Away,Venue,Home_team,Away_team,Score,Home_team_goals,Away_team_goals
Match_day_1,Saturday 10th July 2021,2021-07-10,Premier Sports Cup,A,Balmoor,Peterhead,Heart of Midlothian,0-2,0,2
Match_day_2,Tuesday 13th July 2021,2021-07-13,Premier Sports Cup,H,Tynecastle Park,Heart of Midlothian,Cove Rangers,3-0,3,0
Match_day_3,Saturday 17th July 2021,2021-07-17,Friendly,H,Tynecastle,Heart of Midlothian,Sunderland,0-2,0,2
Match_day_4,Tuesday 20th July 2021,2021-07-20,Premier Sports Cup,A,Forthbank,Stirling Albion,Heart of Midlothian,0-2,0,2
Match_day_5,Sunday 25th July 2021,2021-07-25,Premier Sports Cup,H,Tynecastle Park,Heart of Midlothian,Inverness CT,1-0,1,0


## Convert Home and Away team goals from string to integer data type <a id='convert_data_type' ><a/>
**Step 3** is to convert the home and Away team goals data drom string to integer. This needs to be done so we can use the logical operators greater than, equal to and less than when applying conditional logic to compare the home and away goals in order to determine if it is a win, loss or draw for the mighty Hearts.

In [49]:
# check type of Home_team_goals data
type(df['Home_team_goals'][0])

str

In [50]:
# check output
df.tail()

,Date,Date_formatted,Competition,Home_or_Away,Venue,Home_team,Away_team,Score,Home_team_goals,Away_team_goals
Match_day_35,Wednesday 2nd March 2022,2022-03-02,cinch Premiership,H,Tynecastle Park,Heart of Midlothian,Aberdeen,KO:19:45,KO:19:45,KO:19:45
Match_day_36,Saturday 5th March 2022,2022-03-05,cinch Premiership,A,Tannadice,Dundee United,Heart of Midlothian,KO:15:00,KO:15:00,KO:15:00
Match_day_37,Saturday 19th March 2022,2022-03-19,cinch Premiership,H,Tynecastle Park,Heart of Midlothian,Livingston,KO:15:00,KO:15:00,KO:15:00
Match_day_38,Saturday 2nd April 2022,2022-04-02,cinch Premiership,A,Global Energy Stadium,Ross County,Heart of Midlothian,KO:15:00,KO:15:00,KO:15:00
Match_day_39,Saturday 9th April 2022,2022-04-09,cinch Premiership,H,Tynecastle Park,Heart of Midlothian,Hibernian,KO:15:00,KO:15:00,KO:15:00


## Regualar Expression: use a _Regex_ to remove 'kick off' values in Home team goals, Away team goals and Score <a id='regex' ><a/>
* There is some data in the column that may affect our ability to compare the two columns so we decide to use a _regex_ to remove this unwanted data.
* This needs to be done as otherwise we won't be able to convert Home and Away team goals to integer data type (which we need to do for the logic behind populating the Win/Loss/Draw column) because Python won't allow a non-numeric data type to be converted to integer.

In [51]:
# use a regex to identify the KO time pattern
reg_ex_pattern = re.compile(r'KO:\d{2}:\d{2}')

# replace the regex pattern with '0' and 'TBD' where appropriate
df['Home_team_goals'] = df['Home_team_goals'].str.replace(reg_ex_pattern, '0', regex=True)
df['Away_team_goals'] = df['Away_team_goals'].str.replace(reg_ex_pattern, '0', regex=True)
df['Score'] = df['Score'].str.replace(reg_ex_pattern, 'TBD', regex=True)

# check the output
df.tail()

,Date,Date_formatted,Competition,Home_or_Away,Venue,Home_team,Away_team,Score,Home_team_goals,Away_team_goals
Match_day_35,Wednesday 2nd March 2022,2022-03-02,cinch Premiership,H,Tynecastle Park,Heart of Midlothian,Aberdeen,TBD,0,0
Match_day_36,Saturday 5th March 2022,2022-03-05,cinch Premiership,A,Tannadice,Dundee United,Heart of Midlothian,TBD,0,0
Match_day_37,Saturday 19th March 2022,2022-03-19,cinch Premiership,H,Tynecastle Park,Heart of Midlothian,Livingston,TBD,0,0
Match_day_38,Saturday 2nd April 2022,2022-04-02,cinch Premiership,A,Global Energy Stadium,Ross County,Heart of Midlothian,TBD,0,0
Match_day_39,Saturday 9th April 2022,2022-04-09,cinch Premiership,H,Tynecastle Park,Heart of Midlothian,Hibernian,TBD,0,0


In [52]:
df.head()

,Date,Date_formatted,Competition,Home_or_Away,Venue,Home_team,Away_team,Score,Home_team_goals,Away_team_goals
Match_day_1,Saturday 10th July 2021,2021-07-10,Premier Sports Cup,A,Balmoor,Peterhead,Heart of Midlothian,0-2,0,2
Match_day_2,Tuesday 13th July 2021,2021-07-13,Premier Sports Cup,H,Tynecastle Park,Heart of Midlothian,Cove Rangers,3-0,3,0
Match_day_3,Saturday 17th July 2021,2021-07-17,Friendly,H,Tynecastle,Heart of Midlothian,Sunderland,0-2,0,2
Match_day_4,Tuesday 20th July 2021,2021-07-20,Premier Sports Cup,A,Forthbank,Stirling Albion,Heart of Midlothian,0-2,0,2
Match_day_5,Sunday 25th July 2021,2021-07-25,Premier Sports Cup,H,Tynecastle Park,Heart of Midlothian,Inverness CT,1-0,1,0


In [53]:
# convert Home_team_goals data from string to integer
df['Home_team_goals'] = df['Home_team_goals'].astype(int)
df['Away_team_goals'] = df['Away_team_goals'].astype(int)
type(df['Away_team_goals'][0])

numpy.int64

In [54]:
# check data type of home team goals
type(df['Home_team_goals'][2])

numpy.int64

### Create and populate a win-loss-draw column <a id='win_loss_draw' ><a/>
**Step 4** is to create an empty column and then use conditional logic to populate it...

In [55]:
# create a win/loss/draw column
df['Win/loss/Draw'] = ""
df.head()

,Date,Date_formatted,Competition,Home_or_Away,Venue,Home_team,Away_team,Score,Home_team_goals,Away_team_goals,Win/loss/Draw
Match_day_1,Saturday 10th July 2021,2021-07-10,Premier Sports Cup,A,Balmoor,Peterhead,Heart of Midlothian,0-2,0,2,
Match_day_2,Tuesday 13th July 2021,2021-07-13,Premier Sports Cup,H,Tynecastle Park,Heart of Midlothian,Cove Rangers,3-0,3,0,
Match_day_3,Saturday 17th July 2021,2021-07-17,Friendly,H,Tynecastle,Heart of Midlothian,Sunderland,0-2,0,2,
Match_day_4,Tuesday 20th July 2021,2021-07-20,Premier Sports Cup,A,Forthbank,Stirling Albion,Heart of Midlothian,0-2,0,2,
Match_day_5,Sunday 25th July 2021,2021-07-25,Premier Sports Cup,H,Tynecastle Park,Heart of Midlothian,Inverness CT,1-0,1,0,


### Conditional logic: use _numpy.select_ for determing a win, loss or draw <a id='np_select' ><a/>

In [56]:
# add a condition that checks today's date vs the formatted date and if the date is after today's date then fill the relevant columsn with na
conditions = [
        (df['Home_team'] == 'Heart of Midlothian') & (df['Home_team_goals'] > df['Away_team_goals']),
        (df['Home_team'] == 'Heart of Midlothian') & (df['Home_team_goals'] < df['Away_team_goals']),
        (df['Home_team'] == 'Heart of Midlothian') & (df['Home_team_goals'] == df['Away_team_goals']),
        (df['Home_team'] != 'Heart of Midlothian') & (df['Home_team_goals'] > df['Away_team_goals']),
        (df['Home_team'] != 'Heart of Midlothian') & (df['Home_team_goals'] < df['Away_team_goals']),
        (df['Home_team'] != 'Heart of Midlothian') & (df['Home_team_goals'] == df['Away_team_goals'])
]
    
choices = ['W', 'L', 'D', 'L', 'W', 'D']
    
df['Win/loss/Draw'] = np.select(conditions, choices)

df

,Date,Date_formatted,Competition,Home_or_Away,Venue,Home_team,Away_team,Score,Home_team_goals,Away_team_goals,Win/loss/Draw
Match_day_1,Saturday 10th July 2021,2021-07-10,Premier Sports Cup,A,Balmoor,Peterhead,Heart of Midlothian,0-2,0,2,W
Match_day_2,Tuesday 13th July 2021,2021-07-13,Premier Sports Cup,H,Tynecastle Park,Heart of Midlothian,Cove Rangers,3-0,3,0,W
Match_day_3,Saturday 17th July 2021,2021-07-17,Friendly,H,Tynecastle,Heart of Midlothian,Sunderland,0-2,0,2,L
Match_day_4,Tuesday 20th July 2021,2021-07-20,Premier Sports Cup,A,Forthbank,Stirling Albion,Heart of Midlothian,0-2,0,2,W
Match_day_5,Sunday 25th July 2021,2021-07-25,Premier Sports Cup,H,Tynecastle Park,Heart of Midlothian,Inverness CT,1-0,1,0,W
Match_day_6,Saturday 31st July 2021,2021-07-31,cinch Premiership,H,Tynecastle Park,Heart of Midlothian,Celtic,2-1,2,1,W
Match_day_7,Saturday 7th August 2021,2021-08-07,cinch Premiership,A,St Mirren Park,St Mirren,Heart of Midlothian,1-2,1,2,W
Match_day_8,Sunday 15th August 2021,2021-08-15,Premier Sports Cup,A,Celtic Park,Celtic,Heart of Midlothian,3-2,3,2,L
Match_day_9,Sunday 22nd August 2021,2021-08-22,cinch Premiership,H,Tynecastle Park,Heart of Midlothian,Aberdeen,1-1,1,1,D
Match_day_10,Saturday 28th August 2021,2021-08-28,cinch Premiership,A,Tannadice,Dundee United,Heart of Midlothian,0-2,0,2,W


In [57]:
# check our logic has worked
df.loc[df['Away_team'] == 'Heart of Midlothian']

,Date,Date_formatted,Competition,Home_or_Away,Venue,Home_team,Away_team,Score,Home_team_goals,Away_team_goals,Win/loss/Draw
Match_day_1,Saturday 10th July 2021,2021-07-10,Premier Sports Cup,A,Balmoor,Peterhead,Heart of Midlothian,0-2,0,2,W
Match_day_4,Tuesday 20th July 2021,2021-07-20,Premier Sports Cup,A,Forthbank,Stirling Albion,Heart of Midlothian,0-2,0,2,W
Match_day_7,Saturday 7th August 2021,2021-08-07,cinch Premiership,A,St Mirren Park,St Mirren,Heart of Midlothian,1-2,1,2,W
Match_day_8,Sunday 15th August 2021,2021-08-15,Premier Sports Cup,A,Celtic Park,Celtic,Heart of Midlothian,3-2,3,2,L
Match_day_10,Saturday 28th August 2021,2021-08-28,cinch Premiership,A,Tannadice,Dundee United,Heart of Midlothian,0-2,0,2,W
Match_day_12,Saturday 18th September 2021,2021-09-18,cinch Premiership,A,Global Energy Stadium,Ross County,Heart of Midlothian,2-2,2,2,D
Match_day_15,Saturday 16th October 2021,2021-10-16,cinch Premiership,A,Ibrox,Rangers,Heart of Midlothian,1-1,1,1,D
Match_day_17,Wednesday 27th October 2021,2021-10-27,cinch Premiership,A,McDiarmid Park,St Johnstone,Heart of Midlothian,1-1,1,1,D
Match_day_18,Saturday 30th October 2021,2021-10-30,cinch Premiership,A,Pittodrie,Aberdeen,Heart of Midlothian,2-1,2,1,L
Match_day_20,Saturday 20th November 2021,2021-11-20,cinch Premiership,A,Fir Park,Motherwell,Heart of Midlothian,TBD,0,0,D


### Drop, rename and reorder columns <a id='drop_column' ><a/>
Some of the columns are now no longer necessary, one needs to be renamed and the column order could also be reordered to make it more logical.

In [58]:
# drop the score column
df = df.drop('Score', axis=1)
df

,Date,Date_formatted,Competition,Home_or_Away,Venue,Home_team,Away_team,Home_team_goals,Away_team_goals,Win/loss/Draw
Match_day_1,Saturday 10th July 2021,2021-07-10,Premier Sports Cup,A,Balmoor,Peterhead,Heart of Midlothian,0,2,W
Match_day_2,Tuesday 13th July 2021,2021-07-13,Premier Sports Cup,H,Tynecastle Park,Heart of Midlothian,Cove Rangers,3,0,W
Match_day_3,Saturday 17th July 2021,2021-07-17,Friendly,H,Tynecastle,Heart of Midlothian,Sunderland,0,2,L
Match_day_4,Tuesday 20th July 2021,2021-07-20,Premier Sports Cup,A,Forthbank,Stirling Albion,Heart of Midlothian,0,2,W
Match_day_5,Sunday 25th July 2021,2021-07-25,Premier Sports Cup,H,Tynecastle Park,Heart of Midlothian,Inverness CT,1,0,W
Match_day_6,Saturday 31st July 2021,2021-07-31,cinch Premiership,H,Tynecastle Park,Heart of Midlothian,Celtic,2,1,W
Match_day_7,Saturday 7th August 2021,2021-08-07,cinch Premiership,A,St Mirren Park,St Mirren,Heart of Midlothian,1,2,W
Match_day_8,Sunday 15th August 2021,2021-08-15,Premier Sports Cup,A,Celtic Park,Celtic,Heart of Midlothian,3,2,L
Match_day_9,Sunday 22nd August 2021,2021-08-22,cinch Premiership,H,Tynecastle Park,Heart of Midlothian,Aberdeen,1,1,D
Match_day_10,Saturday 28th August 2021,2021-08-28,cinch Premiership,A,Tannadice,Dundee United,Heart of Midlothian,0,2,W


In [59]:
# rename column (the lower case 'l' on 'loss' looks untidy!)
df = df.rename(columns={'Win/loss/Draw': 'Win/Loss/Draw'})
df.head()

,Date,Date_formatted,Competition,Home_or_Away,Venue,Home_team,Away_team,Home_team_goals,Away_team_goals,Win/Loss/Draw
Match_day_1,Saturday 10th July 2021,2021-07-10,Premier Sports Cup,A,Balmoor,Peterhead,Heart of Midlothian,0,2,W
Match_day_2,Tuesday 13th July 2021,2021-07-13,Premier Sports Cup,H,Tynecastle Park,Heart of Midlothian,Cove Rangers,3,0,W
Match_day_3,Saturday 17th July 2021,2021-07-17,Friendly,H,Tynecastle,Heart of Midlothian,Sunderland,0,2,L
Match_day_4,Tuesday 20th July 2021,2021-07-20,Premier Sports Cup,A,Forthbank,Stirling Albion,Heart of Midlothian,0,2,W
Match_day_5,Sunday 25th July 2021,2021-07-25,Premier Sports Cup,H,Tynecastle Park,Heart of Midlothian,Inverness CT,1,0,W


In [60]:
# reorder columns to make more logical sense
df.columns

Index(['Date', 'Date_formatted', 'Competition', 'Home_or_Away', 'Venue',
       'Home_team', 'Away_team', 'Home_team_goals', 'Away_team_goals',
       'Win/Loss/Draw'],
      dtype='object')

In [61]:
# reorder the columns
columns = ['Date', 'Date_formatted', 'Competition', 'Home_or_Away', 'Venue',
       'Home_team', 'Away_team', 'Home_team_goals', 'Away_team_goals','Win/Loss/Draw',
       ]
df = df[columns]
df.head()

,Date,Date_formatted,Competition,Home_or_Away,Venue,Home_team,Away_team,Home_team_goals,Away_team_goals,Win/Loss/Draw
Match_day_1,Saturday 10th July 2021,2021-07-10,Premier Sports Cup,A,Balmoor,Peterhead,Heart of Midlothian,0,2,W
Match_day_2,Tuesday 13th July 2021,2021-07-13,Premier Sports Cup,H,Tynecastle Park,Heart of Midlothian,Cove Rangers,3,0,W
Match_day_3,Saturday 17th July 2021,2021-07-17,Friendly,H,Tynecastle,Heart of Midlothian,Sunderland,0,2,L
Match_day_4,Tuesday 20th July 2021,2021-07-20,Premier Sports Cup,A,Forthbank,Stirling Albion,Heart of Midlothian,0,2,W
Match_day_5,Sunday 25th July 2021,2021-07-25,Premier Sports Cup,H,Tynecastle Park,Heart of Midlothian,Inverness CT,1,0,W


## Locating fixtures / venues <a id='locate_data' ><a/>

In [62]:
# check a fixture to make sure the data is pulling and showing as it should
df.loc['Match_day_12']

Date               Saturday 18th September 2021
Date_formatted                       2021-09-18
Competition                   cinch Premiership
Home_or_Away                                  A
Venue                     Global Energy Stadium
Home_team                           Ross County
Away_team                   Heart of Midlothian
Home_team_goals                               2
Away_team_goals                               2
Win/Loss/Draw                                 D
Name: Match_day_12, dtype: object

In [63]:
df.loc['Match_day_15', ['Date_formatted', 'Venue', 'Home_team', 'Away_team']]

Date_formatted             2021-10-16
Venue                           Ibrox
Home_team                     Rangers
Away_team         Heart of Midlothian
Name: Match_day_15, dtype: object

In [64]:
df.loc['Match_day_35']

Date               Wednesday 2nd March 2022
Date_formatted                   2022-03-02
Competition               cinch Premiership
Home_or_Away                              H
Venue                       Tynecastle Park
Home_team               Heart of Midlothian
Away_team                          Aberdeen
Home_team_goals                           0
Away_team_goals                           0
Win/Loss/Draw                             D
Name: Match_day_35, dtype: object

The above output of match day 35 (which hasn't been played yet) shows us that matches that haven't been played are showing as being a 0-0 draw. This isn't very useful for analysis purposes so we should tidy this data up...

## Tidy up the data <a id='tbd_data' ><a/>
Change the Win, Loss or Draw and Score data to 'TBD' if the match has not been played

In [65]:
# show matches that haven't been played
df.tail()

,Date,Date_formatted,Competition,Home_or_Away,Venue,Home_team,Away_team,Home_team_goals,Away_team_goals,Win/Loss/Draw
Match_day_35,Wednesday 2nd March 2022,2022-03-02,cinch Premiership,H,Tynecastle Park,Heart of Midlothian,Aberdeen,0,0,D
Match_day_36,Saturday 5th March 2022,2022-03-05,cinch Premiership,A,Tannadice,Dundee United,Heart of Midlothian,0,0,D
Match_day_37,Saturday 19th March 2022,2022-03-19,cinch Premiership,H,Tynecastle Park,Heart of Midlothian,Livingston,0,0,D
Match_day_38,Saturday 2nd April 2022,2022-04-02,cinch Premiership,A,Global Energy Stadium,Ross County,Heart of Midlothian,0,0,D
Match_day_39,Saturday 9th April 2022,2022-04-09,cinch Premiership,H,Tynecastle Park,Heart of Midlothian,Hibernian,0,0,D


In [66]:
# get today's date
date.today()

datetime.date(2021, 10, 30)

In [67]:
# use np.select to change the 0 goals and 'D' values to 'TBD' for games that haven't been played yet
# use datetime to apply logic that says if the date in the date column is after today's date then place TBD in the home and away goals and score columns
condition = [df['Date_formatted'] >= date.today()]
choice = ['TBD']
    
df['Home_team_goals'] = np.select(condition, choice, default=df['Home_team_goals'])
df['Away_team_goals'] = np.select(condition, choice, default=df['Away_team_goals'])
df['Win/Loss/Draw'] = np.select(condition, choice, default=df['Win/Loss/Draw'])

df

,Date,Date_formatted,Competition,Home_or_Away,Venue,Home_team,Away_team,Home_team_goals,Away_team_goals,Win/Loss/Draw
Match_day_1,Saturday 10th July 2021,2021-07-10,Premier Sports Cup,A,Balmoor,Peterhead,Heart of Midlothian,0,2,W
Match_day_2,Tuesday 13th July 2021,2021-07-13,Premier Sports Cup,H,Tynecastle Park,Heart of Midlothian,Cove Rangers,3,0,W
Match_day_3,Saturday 17th July 2021,2021-07-17,Friendly,H,Tynecastle,Heart of Midlothian,Sunderland,0,2,L
Match_day_4,Tuesday 20th July 2021,2021-07-20,Premier Sports Cup,A,Forthbank,Stirling Albion,Heart of Midlothian,0,2,W
Match_day_5,Sunday 25th July 2021,2021-07-25,Premier Sports Cup,H,Tynecastle Park,Heart of Midlothian,Inverness CT,1,0,W
Match_day_6,Saturday 31st July 2021,2021-07-31,cinch Premiership,H,Tynecastle Park,Heart of Midlothian,Celtic,2,1,W
Match_day_7,Saturday 7th August 2021,2021-08-07,cinch Premiership,A,St Mirren Park,St Mirren,Heart of Midlothian,1,2,W
Match_day_8,Sunday 15th August 2021,2021-08-15,Premier Sports Cup,A,Celtic Park,Celtic,Heart of Midlothian,3,2,L
Match_day_9,Sunday 22nd August 2021,2021-08-22,cinch Premiership,H,Tynecastle Park,Heart of Midlothian,Aberdeen,1,1,D
Match_day_10,Saturday 28th August 2021,2021-08-28,cinch Premiership,A,Tannadice,Dundee United,Heart of Midlothian,0,2,W


------------------------------------------------
# Export to _.csv_ <a id='export_csv' ><a/>
------------------------------------------------
The next task is to export the data frame to a _.csv_ file so that it is easy to use for stakeholders.

In [68]:
# export the dataframe to csv
df.to_csv('hearts_fixtures_results.csv')
# df.to_excel('hearts_fixtures_results.xlsx')

-----------------------------------
# Exploring the dataframe and descriptives of season so far... <a id='explore' ><a/>
-----------------------------------

In [70]:
# get basic info on the data fram
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39 entries, Match_day_1 to Match_day_39
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Date             39 non-null     object
 1   Date_formatted   39 non-null     object
 2   Competition      39 non-null     object
 3   Home_or_Away     39 non-null     object
 4   Venue            39 non-null     object
 5   Home_team        39 non-null     object
 6   Away_team        39 non-null     object
 7   Home_team_goals  39 non-null     object
 8   Away_team_goals  39 non-null     object
 9   Win/Loss/Draw    39 non-null     object
dtypes: object(10)
memory usage: 4.4+ KB


In [73]:
# check for null values
df.isnull().sum()

Date               0
Date_formatted     0
Competition        0
Home_or_Away       0
Venue              0
Home_team          0
Away_team          0
Home_team_goals    0
Away_team_goals    0
Win/Loss/Draw      0
dtype: int64

In [ ]:
# if there were null values in one column we could replace them with the following code
# e.g. there were null values in the 'Competition' column because friendly matches hadn't been classified as a specific competition
# df['Competition'].fillna('Friendly', inplace=True)

In [77]:
# let's see how many wins, losses and draws Hearts have had
df['Win/Loss/Draw'].value_counts()

TBD    22
W       9
D       6
L       2
Name: Win/Loss/Draw, dtype: int64

In [119]:
# count the wins
df['Win/Loss/Draw'].value_counts()['W']

9

In [84]:
# the above output includes pre season cup games - we want to find out how many wins, losses and draws Hearts have had in the league
# create new dataframe that only has the cinch Premiership games in it
df_league = df[df.Competition == 'cinch Premiership']
# count the league wins, losses and draws
df_league['Win/Loss/Draw'].value_counts()

TBD    22
D       6
W       5
Name: Win/Loss/Draw, dtype: int64

In [86]:
df_league.head()

,Date,Date_formatted,Competition,Home_or_Away,Venue,Home_team,Away_team,Home_team_goals,Away_team_goals,Win/Loss/Draw
Match_day_6,Saturday 31st July 2021,2021-07-31,cinch Premiership,H,Tynecastle Park,Heart of Midlothian,Celtic,2,1,W
Match_day_7,Saturday 7th August 2021,2021-08-07,cinch Premiership,A,St Mirren Park,St Mirren,Heart of Midlothian,1,2,W
Match_day_9,Sunday 22nd August 2021,2021-08-22,cinch Premiership,H,Tynecastle Park,Heart of Midlothian,Aberdeen,1,1,D
Match_day_10,Saturday 28th August 2021,2021-08-28,cinch Premiership,A,Tannadice,Dundee United,Heart of Midlothian,0,2,W
Match_day_11,Sunday 12th September 2021,2021-09-12,cinch Premiership,H,Tynecastle Park,Heart of Midlothian,Hibernian,0,0,D


In [87]:
# reset the df_league index
# create Match day 1-39
league_game = []
for i in range(0, len(df_league['Date'])):
    league_game.append(f"League_game_{i+1}")

league_game

['League_game_1',
 'League_game_2',
 'League_game_3',
 'League_game_4',
 'League_game_5',
 'League_game_6',
 'League_game_7',
 'League_game_8',
 'League_game_9',
 'League_game_10',
 'League_game_11',
 'League_game_12',
 'League_game_13',
 'League_game_14',
 'League_game_15',
 'League_game_16',
 'League_game_17',
 'League_game_18',
 'League_game_19',
 'League_game_20',
 'League_game_21',
 'League_game_22',
 'League_game_23',
 'League_game_24',
 'League_game_25',
 'League_game_26',
 'League_game_27',
 'League_game_28',
 'League_game_29',
 'League_game_30',
 'League_game_31',
 'League_game_32',
 'League_game_33']

In [88]:
# set the axis for the new data frame
df_league = df_league.set_axis(labels=league_game, axis=0)
df_league.head()

,Date,Date_formatted,Competition,Home_or_Away,Venue,Home_team,Away_team,Home_team_goals,Away_team_goals,Win/Loss/Draw
League_game_1,Saturday 31st July 2021,2021-07-31,cinch Premiership,H,Tynecastle Park,Heart of Midlothian,Celtic,2,1,W
League_game_2,Saturday 7th August 2021,2021-08-07,cinch Premiership,A,St Mirren Park,St Mirren,Heart of Midlothian,1,2,W
League_game_3,Sunday 22nd August 2021,2021-08-22,cinch Premiership,H,Tynecastle Park,Heart of Midlothian,Aberdeen,1,1,D
League_game_4,Saturday 28th August 2021,2021-08-28,cinch Premiership,A,Tannadice,Dundee United,Heart of Midlothian,0,2,W
League_game_5,Sunday 12th September 2021,2021-09-12,cinch Premiership,H,Tynecastle Park,Heart of Midlothian,Hibernian,0,0,D


In [118]:
# calculate the win % in the league
league_win_percentage = df_league['Win/Loss/Draw'].value_counts()['W'] / (df_league['Win/Loss/Draw'].value_counts()['W'] # + (df_league['Win/Loss/Draw'].value_counts()['L'])
                                                                          + df_league['Win/Loss/Draw'].value_counts()['D']) * 100
print(f"Win % = {league_win_percentage.round(1)}%")

Win % = 45.5%
